In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import re
import itertools
from tqdm import tqdm

from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score,recall_score,roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
import networkx as nx

import lightgbm as lgb

In [4]:
train_data = pd.read_csv('/Users/s0c02nj/Desktop/Personal Edu doc/Competitions /Hike/train/train.csv')
train_feat = pd.read_csv('/Users/s0c02nj/Desktop/Personal Edu doc/Competitions /Hike/train/user_features.csv')

In [5]:
test_data = pd.read_csv('/Users/s0c02nj/Desktop/Personal Edu doc/Competitions /Hike/test.csv')

In [6]:
train_data.shape

(70661802, 3)

Stratified sample

In [10]:
train_data_sample = train_data.sample(n= 706618, weights='is_chat', random_state=1).reset_index(drop=True)


In [7]:

#X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42, stratify=y)

All nodes are present

In [ ]:
train_data1 = train_data_new[['node1_id']]
train_data2 = train_data_new[['node2_id']]

y_train = train_data_new['is_chat']

In [ ]:
test_data1 = test_data[['node1_id']]
test_data2 = test_data[['node2_id']]


In [ ]:
train_feat1 = pd.merge(left = train_data1, right= train_feat, how='left', 
                 left_on=['node1_id'], right_on=['node_id'])

In [ ]:
train_feat2 = pd.merge(left = train_data2, right= train_feat, how='left', 
                 left_on=['node2_id'], right_on=['node_id'])

In [ ]:
test_feat1 = pd.merge(left = test_data1, right= train_feat, how='left', 
                 left_on=['node1_id'], right_on=['node_id'])

In [ ]:
test_feat2 = pd.merge(left = test_data2, right= train_feat, how='left', 
                 left_on=['node2_id'], right_on=['node_id'])

In [ ]:
cols_std = ['f1','f2','f3','f4','f5','f6','f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13']

In [ ]:
path = '/Users/s0c02nj/Desktop/Hike/node_vec/node2vec.txt'

In [ ]:
count = 0
vectors_list = []

for o in tqdm(open(path)):
    
    values = o.split()
    values[0] = int(values[0])
    values[1:] = list(map(float, values[1:]))
    
    vectors_list.append(values)

In [ ]:
cols = []
for i in range(0,16):
    cols.append('data'+str(i))

In [ ]:
cols = ['node_id']+ cols

In [ ]:
node_vec = pd.DataFrame(vectors_list)
node_vec.columns = cols

In [ ]:
node_vec = node_vec.dropna()

In [ ]:
node_vec.index = range(0,len(node_vec))

In [ ]:
train_feat1_fin = pd.merge(left = train_feat1, right= node_vec, how='left', 
                 left_on=['node1_id'], right_on=['node_id'])

In [ ]:
train_feat2_fin = pd.merge(left = train_feat2, right= node_vec, how='left', 
                 left_on=['node2_id'], right_on=['node_id'])

In [ ]:
test_feat1_fin = pd.merge(left = test_feat1, right= node_vec, how='left', 
                 left_on=['node1_id'], right_on=['node_id'])

In [ ]:
test_feat2_fin = pd.merge(left = test_feat2, right= node_vec, how='left', 
                 left_on=['node2_id'], right_on=['node_id'])

In [ ]:
cols = []
for i in range(0,16):
    cols.append('data'+str(i))
cols_total = cols_std + cols

In [ ]:
train_feat1_fin = train_feat1_fin[cols_total]
train_feat2_fin = train_feat2_fin[cols_total]

In [ ]:
test_feat1_fin = test_feat1_fin[cols_total]
test_feat2_fin = test_feat2_fin[cols_total]

In [ ]:
x_train = pd.concat([train_feat1_fin,train_feat2_fin],axis=1)

In [ ]:
x_test = pd.concat([test_feat1_fin,test_feat2_fin],axis=1)

Random forest

In [ ]:
x1_train, x1_val, y1_train, y1_val = train_test_split(x_train, y_train,test_size=0.2,random_state=0)

In [ ]:
rf =  RandomForestClassifier(bootstrap=True, criterion='gini',
                max_depth=20, max_features='auto', max_leaf_nodes=None, 
                min_impurity_decrease=0.0, min_impurity_split=None,
                min_samples_leaf=1, min_samples_split=2,
                min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=3,
                oob_score=False, random_state=12, warm_start=False,
                class_weight={0:5,1:70}, verbose=1)

In [ ]:
rf.fit(x1_train, y1_train)

In [ ]:
print (rf.score(x1_val, y1_val))
print (recall_score(y1_val, rf.predict(x1_val)))
print (precision_score(y1_val, rf.predict(x1_val)))
print (roc_auc_score(y1_val, rf.predict(x1_val)))

In [ ]:
rf_pred = rf.predict_proba(x_test)

In [ ]:
y_ones = rf_pred[:,1]

Ligh gbm

In [ ]:
random_state = 42
np.random.seed(random_state)

lgb_params = {
    "objective" : "binary",
    "metric" : "auc",
    "boosting": 'gbdt',
    "max_depth" : 20,
    "num_leaves" : 20,
    "learning_rate" : 0.01,
    "bagging_freq": 5,
    "bagging_fraction" : 0.4,
    "feature_fraction" : 0.05,
    "min_data_in_leaf": 80,
    "min_sum_heassian_in_leaf": 10,
    "tree_learner": "serial",
    "boost_from_average": "false",
    #"lambda_l1" : 5,
    #"lambda_l2" : 5,
    "bagging_seed" : random_state,
    "verbosity" : 1,
    "seed": random_state
}

In [ ]:
trn_data = lgb.Dataset(x1_train, label= y1_train)
val_data = lgb.Dataset(x1_val, label= y1_val)

In [ ]:
lgb_clf = lgb.train(lgb_params,
                    trn_data,
                    9000,
                    valid_sets = [trn_data, val_data],
                    early_stopping_rounds=3000,
                    verbose_eval = 1000,
                    )

In [ ]:
pred_valid = lgb_clf.predict(x1_val)
val_score =  roc_auc_score(y1_val, pred_valid)

In [ ]:
y_pred = lgb_clf.predict(x_test)

In [ ]:
#?lgb_clf.predict
#y_pred

In [ ]:
sub_df = test_data.copy()
sub_df['is_chat'] = y_pred
sub_df = sub_df[['id','is_chat']]

In [ ]:
z = np.load('/Users/s0c02nj/Desktop/Hike/same_node_test_ids.npy')
z_set = set(z)

In [ ]:
out = []
for i in tqdm(range(0,len(sub_df))):
    
    i_id = sub_df['id'].iloc[i]
    
    if i_id in z_set:
        k=0
    else:
        k = sub_df['is_chat'].iloc[i]
        
    out.append(k)

In [ ]:
sub_df['is_chat'] = out

In [ ]:
sub_df.to_csv('/Users/s0c02nj/Desktop/Hike/Submission7.csv',index=False)